In [24]:
# 导入数据的函数
import numpy as np


def loadSimpData():
    datMat=np.matrix([
        [1.,2.1],
        [2.,1.1],
        [1.3,1.],
        [1.,1.],
        [2.,1.]
    ])
    classLabels=[1.,1.,-1.,-1.,1.]
    return datMat,classLabels

### 单层决策树的构建
#### 是通过暴力穷举生成对当前数据集权重的最优决策树

In [25]:
def stumpClassify(dataMatrix, dimen, threshVal, threshIneq):
    """
    似乎这是一个基于单特征的分类器
    dimen:特征,维度,第几列
    threshVal:阈值
    threshIneq:阈值不等号:('lt','gt')
    """
    retArray = np.ones((dataMatrix.shape[0], 1))    # 要返回的预测值
    if threshIneq == 'lt':
        retArray[dataMatrix[:, dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:, dimen] > threshVal] = -1.0
    return retArray


def buildStump(dataArr, classLabels, D):
    """
    构造单层决策树
    D:数据的权重向量
    return: 最优的单层决策树
    """
    # 初始化
    dataMatrix = np.mat(dataArr)
    labelMat = np.mat(classLabels).T
    m, n = dataMatrix.shape
    numsteps = 10.0    # 用于在特征的所有可能值上进行遍历
    bestStump = {}   # 用于保存最优决策树
    bestClassEst = np.mat(np.zeros((m, 1)))
    minError = np.inf   # 初始化最小误差

    for i in range(n):    # 遍历每个特征
        rangeMin = dataMatrix[:, i].min()    # 特征的最小值
        rangeMax = dataMatrix[:, i].max()
        stepSize = (rangeMax-rangeMin)/numsteps    # 步幅
        for j in range(-1, int(numsteps)+1):
            for inequal in ['lt', 'gt']:    # 对每个不等号
                # 对阈值从rangeMin遍历到rangeMax
                threshVal = rangeMin+float(j)*stepSize
                predictedVals = stumpClassify(
                    dataMatrix, i, threshVal, inequal)
                errArr = np.mat(np.ones((m, 1)))    # 误差，列向量，初始化为全部预测错误，故全1
                errArr[predictedVals == labelMat] = 0   # 预测对的置为0
                # 计算加权错误率
                weightedError = D.T*errArr    # 这个函数中，样本的权重D用来计算误差，
                # 由于错误的样本对应的权重更大，且对应的errArr为1，所以会导致weightedError更大，
                # 使得下面的if语句不易进行，从而进行更多的循环，进行更细致的划分策略。

                if weightedError < minError:
                    minError = weightedError    # 更新最小误差
                    bestClassEst = predictedVals.copy()   # 更新最优预测结果。
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump, minError, bestClassEst


D = np.mat(np.ones((5, 1))/5)    # 初始化权重
datMat, classLabels = loadSimpData()
buildStump(datMat, classLabels, D)


({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'},
 matrix([[0.2]]),
 array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

### 7.4 完整AdaBoost算法的实现
#### 程序清单7-2 基于单层决策树的AdaBoost训练过程

In [26]:
def adaBoostTrainDS(dataArr, classLabels, numIter=40):
    """
    numIter: number of iterations,也是弱学习器的个数
    返回弱分类器列表及其对应的权重
    """
    # 初始化
    weakClassArr = []    # 弱学习器列表
    m = dataArr.shape[0]
    D = np.mat(np.ones((m, 1)))/m    # 初始化各个样本的权重
    aggClassEst = np.mat(np.zeros((m, 1)))    # 当前弱学习器的加权预测值

    for i in range(numIter):
        bestStump, error, classEst = buildStump(dataArr, classLabels, D)
        # print('各个样本的权重: ', D.T)
        # 计算弱学习器的权重alpha同时防止除以0溢出
        alpha = float(0.5*np.log((1-error)/max(error, 1e-16)))
        bestStump['alpha'] = alpha
        weakClassArr.append(bestStump)
        # 更新各个样本的权重
        expon = np.multiply(-1*alpha*np.mat(classLabels).T, classEst)
        D = np.multiply(D, np.exp(expon))
        D = D/D.sum()    # 归一

        aggClassEst += alpha*classEst    # 加权预测值
        print('预测结果: ', np.sign(aggClassEst).T)
        aggErrors = np.multiply(np.sign(aggClassEst)!=np.mat(classLabels).T,np.ones((m,1)))    # 010100011的形式
        errorRate=aggErrors.sum()/m
        print('总错误率：',errorRate)
        if errorRate==0:
            break    # 提前训练完成，则直接提前退出
    return weakClassArr


classfierArr=adaBoostTrainDS(datMat, classLabels,10)
classfierArr

预测结果:  [[-1.  1. -1. -1.  1.]]
总错误率： 0.2
预测结果:  [[ 1.  1. -1. -1. -1.]]
总错误率： 0.2
预测结果:  [[ 1.  1. -1. -1.  1.]]
总错误率： 0.0


[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565},
 {'dim': 0, 'thresh': 0.9, 'ineq': 'lt', 'alpha': 0.8958797346140273}]

### 7.5 测试算法：基于AdaBoost的分类
#### 程序清单7-3 AdaBoost分类函数

In [27]:
def adaClassify(datToClass, classifyArr):
    """
    datToClass:要分类的样本
    classifier:训练好的ada分类器
    return:+-1 标签
    """
    dataMatrix=np.mat(datToClass)
    m=dataMatrix.shape[0]
    aggClassEst=np.mat(np.zeros((m,1)))    # 累计预测结果（加权）
    for i in range(len(classifyArr)):
        classEst=stumpClassify(dataMatrix,
                               dimen=classifyArr[i]['dim'],
                               threshVal=classifyArr[i]['thresh'],
                               threshIneq=classifyArr[i]['ineq'])
        aggClassEst+=classEst*classfierArr[i]['alpha']
        print('第%d个弱分类器的预测结果:'%i,classEst)
        print('本次迭代的结果（未符号函数作用时）',aggClassEst)
        print('————————————————————————————————————————————————————')
    return np.sign(aggClassEst)


adaClassify([0,0],classfierArr)

# 由aggClassEst可以看出，随着迭代次数的进行，aggClassEst的值愈发趋向更大的负值，即分类结果越来越强

第0个弱分类器的预测结果: [[-1.]]
本次迭代的结果（未符号函数作用时） [[-0.69314718]]
————————————————————————————————————————————————————
第1个弱分类器的预测结果: [[-1.]]
本次迭代的结果（未符号函数作用时） [[-1.66610226]]
————————————————————————————————————————————————————
第2个弱分类器的预测结果: [[-1.]]
本次迭代的结果（未符号函数作用时） [[-2.56198199]]
————————————————————————————————————————————————————


matrix([[-1.]])

### 7.6 示例：在一个难数据集上应用AdaBoost
#### 使用病马数据集

In [28]:
def loadDataSet(fileName):
    numFeat=len(open(fileName).readline().split('\t'))
    dataMat=[]
    labelMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        lineArr=[]
        curLine=line.strip().split('\t')
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return np.matrix(dataMat),labelMat


dataPath='D:\\机器学习实战代码\\machinelearninginaction\\Ch07\\horseColicTraining2.txt'
train_dat,train_label=loadDataSet(dataPath)


# 训练
classifierArray=adaBoostTrainDS(train_dat,train_label,10)

预测结果:  [[-1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1. -1. -1.  1.
   1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1. -1.
   1. -1.  1.  1. -1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.  1.
   1.  1.  1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1. -1.  1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1. -1.  1.  1.  1.
   1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.
  -1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1. -1.  1. -1.  1.  1.
  -1. -1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1.  1. -1.  1.  1.
   1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
  -1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1. -1. -1. -1.
   1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.
   1.  1. -1.  1.  1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1. -1.  1.  1.  1.
  -1.  1.  1.  1. -1. -1.  1. -